# Sniffing Procedure Script

**This script can be used to run several experiments in succession by reading an excel file stored in the "Experiment Files" folder.**

The script returns the following: 
* An entry in the database file with: Experiment ID, timestamp, User, Procedure ID, Odor ID, locations of PNG and CSV Files, and additional notes
* The CSV files containing the raw data for the full spectrum (including the 5 minute nitrogen purge), experiment spectrum, phase, and phase derivative
* The analysis images of the fourier transform, phase, and phase derivative

## Setup

* Import relevant modules. sniffing_functions.py is the file containing additional functions written for this script
* Establish communication with spectrometer (connected via USB)
* Establish serial communication with teensy (usually COM4 or COM5) - check device manager for the correct COM port
* Set integration time - check the current intensity values in the Spectrasuite software. Choose an integration time where the reflectance is not saturated. Usually around 3-7 ms.

In [1]:
from seabreeze.spectrometers import Spectrometer # opens communication and control with spectrometer
from matplotlib import pyplot as plt
import time
import numpy as np
import serial
import sys
import numpy as np
import csv
from sniffing_functions import * # functions built for this project
import pandas as pd
import sqlite3
import datetime
from scipy.signal import savgol_filter as sgf
from math import factorial as fact
from tkinter import Tk, filedialog
from IPython.display import clear_output

## INITIALIZE COMMUNICATIONS

# Spectrometer
try: 
    spec = Spectrometer.from_first_available() # Start communication with spectrometer
except Exception as e: 
    print(e)
    print("Please unplug and plug in the spectrometer, then restart and clear outputs for this script.")
    
time_in_ms = int(input("Set integration time (in ms): "))*10**3
spec.integration_time_micros(time_in_ms) # Sets integration time


# SQLite Databse
conn = sqlite3.connect('Database/Sniffing-Sensor.db') # connect to the database
c = conn.cursor() # Creates a cursor to interact with database

# Teensy
port = input("Which port number is is the teensy?: ").strip()
ser = serial.Serial("COM{}".format(port), 9600) # Opening serial port to communicate with teensy

Set integration time (in ms): 2
Which port number is is the teensy?: 5


### Run this cell between experiments

**In between experiments, replace all the falcon tubes with the empty tubes labeled 'N' and run the cell below to purge the tubing of all residual chemicals**

In [2]:
ser.write(b'ABCD')
time.sleep(120)
ser.write(b'abcd')

4

### Run this cell if an error occurs and the valves are still open

**This will close the valves**

In [ ]:
ser.write(b'abcd')

## Procedure

When you run the following cell, you will be prompted to select an experiment file. Choose an excel document containing rows which specify the: sniffing procedure (must be identical, case-sensitive to the sniffing procedure found in the database), and chemical (must be identical, case-sensitive to the odor found in the database). 

**Before you run the cell, make sure that you have changed the "valves" dictionary so that your chemicals correspond to the correct valves**

For instance, {"Ethanol": "B", "Water": "C", "IPA": "D"} can be changed to {"Pentane": "B", "Hexane": "C", "Heptane": "D"} if your experiment deals with alkanes.

In [ ]:
# Allow user to choose the experiment file
root = Tk()
root.fileName = filedialog.askopenfilename( filetypes = ( ("Excel File", "*.xlsx"), ("All Files", "*.*") ) )

# Read excel and convert to pandas dataframe
experiment = pd.read_excel('{}'.format(root.fileName))
experiment.set_index("True Runs", inplace=True) # indexed by the True run number
experiment.dropna(axis=0, how="all", inplace=True) #removes all NaN rows

num_exp = len(experiment.index) # create an entry for each experiment run

# ser.write(b'A')
# time.sleep(300)
# ser.write(b'a')

# Indexing starts at zero
for run in range(num_exp):
    clear_output() # clears output after every run
    
    # --------------- DESIGNING RECIPE -------------------
    
    # Experiment Name - Integer ID
    exp_name = "Default"
    c.execute("SELECT ExperimentID FROM Experiments")
    data = c.fetchall()
    if data == []: 
        exp_name = 1
    else:
        exp_name = data[-1][0] + 1
        
    # Timestamp
    timestamp = str(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
        
    # Compound name coded into its OdorID from the database
    try: 
        compound = list(experiment['Odorant'])[run]
        temp = c.execute("SELECT OdorID FROM Odors WHERE Name='%s'" %str(compound.upper())) # All database entries are uppercase
        OdorID = c.fetchall()[0][0]
        print("Run Number: {}\nExperiment Name: {}\nCompound: {}".format(run+1, exp_name, compound))
    except Exception as e: 
        print("Compound Naming Error: {}".format(e))
        
    # Selecting procedure
    try: 
        procedure = list(experiment['Procedure'])[run]
        c.execute("SELECT ProcedureID FROM Procedures WHERE Name='%s'" %str(procedure.capitalize()))
        ProcedureID = c.fetchall()[0][0]
        c.execute("SELECT Procedure FROM Procedures WHERE Name='%s'" %str(procedure.capitalize()))
        data = c.fetchall()[0][0]
        proc = dict([i.split(":") for i in data.split("\n")])
        
    except Exception as e: 
        print("Procedure Naming Error: {}".format(e))

    # Creates a PNG and CSV File pointing to the location of the files
    PNG = "../Scan CSV Files/PNG/{}.png".format(exp_name)
    CSV = "../Scan CSV Files/{}.png".format(exp_name)


    # ======================= WRITING TO CSV File =======================

    # The relevant data of the experiment
    df = pd.DataFrame() # creates a new dataframe for csv file
    df["Wavelengths"] = spec.wavelengths()
    df.set_index("Wavelengths")
    
    # Spectra data including Nitrogen purge (first 5 min)
    df_broad = pd.DataFrame() # creates a new dataframe for csv file
    df_broad["Wavelengths"] = spec.wavelengths()
    df_broad.set_index("Wavelengths")

    # ============================== RECIPE ==============================
    
    delay_time = 20 # scans per second

    intensities_broad = [spec.intensities()] # empty list to be populated with intensity values from spec
    elapse_broad = [0] #empty list to be populated with time values

    valves = {"Ethanol": "B",
              "Water":   "C", 
              "IPA":     "D"} # edit the chemicals and the valve numbers for your experiment
    
    try: 
        purge_time = 300 # seconds
        print("Cleaning out chamber. Please standby for 5 minutes.")
        Purge(purge_time, ser, intensities_broad, spec, elapse_broad)

        intensities = [spec.intensities()]
        elapse = [0]
    
        if ProcedureID == 2: 
            short_sniff(proc, valves, compound, ser, intensities, spec, delay_time, elapse)
        if ProcedureID == 3:
            deep_sniff(proc, valves, compound, ser, intensities, spec, delay_time, elapse)
        if ProcedureID == 4:
            short_held_sniff(proc, valves, compound, ser, intensities, spec, delay_time, elapse)
        if ProcedureID == 5:
            short_sniff_exhale(proc, valves, compound, ser, intensities, spec, delay_time, elapse)
            
    finally:
        ser.write(b'abcd') ## If for some reason, there is a communication error, the valves will be turned off.
    
    for i in range(len(elapse_broad)):
        df_broad["{}".format(round(elapse_broad[i],2))] = intensities_broad[i]
                 
    for i in range(len(elapse)):
        df["{}".format(round(elapse[i],2))] = intensities[i] # Write to the i'th column the intensities
        df_broad["{}".format(round(elapse[i],2)+round(elapse_broad[-1],2))] = intensities[i]

    sys.stdout.write("\n{}\nDone Captures!\n".format('-'*30))

    # ========================== WRITING TO FILES ==========================
    print("Saving to CSV in ../Scan CSV Files/{}.csv".format(exp_name))
    df.to_csv("../Scan CSV Files/{}.csv".format(exp_name)) # Writing to the CSV file
    df_broad.to_csv("../Scan CSV Files/Full Spectra/{}.csv".format(exp_name))

    plot_river(np.array(intensities),plt, np) # Plotting river plot
    plt.savefig("../Scan CSV Files/PNG/{}.png".format(exp_name)) # Save figure to file
    plt.close()

    Notes = "Procedure {}".format(ProcedureID)

    c.execute("""INSERT INTO Experiments (Timestamp, OdorID, ProcedureID, PNG, CSV, Notes)
                 VALUES (?, ?, ?, ?, ?, ?)""",
                 (timestamp, OdorID, ProcedureID, PNG, CSV, Notes))
    
    conn.commit() # Save values into SQLite

    # ========================== DATA ANALYSIS ==========================
    print("Analyzing data and saving plots.")
                 
    # Read file and convert it to pandas dataframe
    df = pd.read_csv("../Scan CSV Files/{}.csv".format(exp_name))
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.set_index("Wavelengths", inplace=True)
    phase_name = exp_name
    exp_name = "{}".format(exp_name) + compound
    
    
    ## FOURIER TRANSFORM
    fig = plt.figure(figsize=(14,7))
    ft = fig.add_subplot(111, title="{}: Fourier transform".format(exp_name), xlabel="time (s)")

    sliced = [df.index[500], df.index[1200]] # Wavelengths between 400 and 900

    # taking the transpose means you get the fourier transform with time axis
    ft_data = np.fft.fft( (df.loc[sliced[0]:sliced[1], :]).transpose() ) 

    ft.plot(np.real(ft_data[:,1]))
    ft.plot(np.imag(ft_data[:,1]))

    # axes titles
    ft.set_xticks(np.linspace(0, len(df.columns), 11)) # 11 ticks
    ft.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%(int(len(df.columns)/10)) == 0]) 
    plt.savefig("../Analysis Images/{}_fourier.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE CALCULATION
    R = np.real(ft_data[:, 1])
    I = np.imag(ft_data[:, 1])

    # Calculating Phase
    phase = I / (R ** 2 + I ** 2) ** 0.5
    phase = sgf(phase, window_length=31, polyorder=3)

    # Normalization
    phase = phase/np.sqrt(np.sum(phase**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))

    ph = fig.add_subplot(111, title="{}: Phase".format(exp_name), xlabel="time (s)")

    ph.plot(phase)

    ph.set_xticks(np.linspace(0, len(df.columns), 11))
    ph.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])
    
    print("Saving Phase to CSV in ../Scan CSV Files/Phase/{}.csv".format(phase_name))  
    phase_df = pd.DataFrame(phase)
    phase_df.to_csv("../Scan CSV Files/Phase/{}.csv".format(phase_name))

    plt.savefig("../Analysis Images/{}_phase.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE DERIVATIVE CALCULATION
    phase_deriv = np.diff(phase)
    phase_deriv = sgf(phase_deriv, window_length=31, polyorder=3)

    # Normalization
    phase_deriv = phase_deriv/np.sqrt(np.sum(phase_deriv**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), xlabel="time (s)")

    pder.plot(phase_deriv)

    pder.set_xticks(np.linspace(0, len(df.columns), 11))
    pder.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    pd_df = pd.DataFrame(phase_deriv)
    pd_df.to_csv("../Scan CSV Files/Phase Derivative/{}.csv".format(exp_name))
    
    plt.savefig("../Analysis Images/{}_phase_deriv.png".format(exp_name))
    plt.close(fig)
    
    print("Saved all plots in ../Analysis Images/, and output {}.xlsx to ../Experiment Files/".format(exp_name))
    
ser.close() # Close serial connection
c.close() # Close cursor 
conn.close() # Close connection to save memory